In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os, sys, shutil, bz2, random, resource, warnings, subprocess, copy, re, glob, json
from pathlib import Path
import xml.etree.ElementTree as ET
from subprocess import Popen, PIPE, STDOUT
import pandas as pd
pd.set_option('display.max_columns', 50)
import numpy as np
from Bio import SeqIO, bgzf
from Bio.SeqIO.QualityIO import FastqGeneralIterator
from Bio import Seq, SeqIO, SearchIO, SeqRecord
from mpire import WorkerPool
import jellyfish


In [3]:
# Navigate back to NBdir in case of re-running a code block:
if not 'NBdir' in globals():
    NBdir = os.getcwd()
print('Notebook is in: {}'.format(NBdir))
os.chdir(NBdir)  # If you changed the current working dir, this will take you back to the notebook dir.

# Define the path to the repo folder.
# Change if necessary.
homedir = '/'.join(NBdir.split('/')[0:-2])
print('Repo is in: {}'.format(homedir))
sys.path.insert(1, homedir)
from utils.read_processing import index_to_sample_df, downsample_raw_input, AR_merge, BC_split, Kmer_analysis, BC_analysis, UMI_trim
from utils.alignment import SWIPE_align

# These are default folder names for data and raw fastq files
# relative to the folder in which this notebook is in:
data_folder = 'data'
seq_folder = 'raw_fastq'
seq_folder_noDS = seq_folder # Not downsampled

# These folder names are used in subsequent processing steps
# to dump data. Best to not change:
AdapterRemoval_dir = 'AdapterRemoval'
BC_dir = 'BC_split'
UMI_dir = 'UMI_trimmed'
align_dir = 'SWalign'
tRNA_database = dict()
tRNA_database['human'] = '{}/utils/tRNA_database/human/hg38-tRNAs.fa'.format(homedir)
tRNA_database['mouse'] = '{}/utils/tRNA_database/mouse/mm10-tRNAs.fa'.format(homedir)
SWIPE_score_mat = '{}/utils/nuc_score-matrix.txt'.format(homedir)

# Define minimum read length based on minimum insert size:
MIN_INSERT_LEN = 20
UMI_LEN = 10
BC_MAX_LEN = 19
MIN_READ_LEN = MIN_INSERT_LEN + UMI_LEN + BC_MAX_LEN
print('Using minimum read length: {} (after merge)'.format(MIN_READ_LEN))

# The minimum alignment score.
# The alignment score is calculated as 1 per match,
# -2 per mismatch, -6 per gap opening and -1 per gap extension.
MIN_SCORE_ALIGN = 20
print('Using minimum alignemnt score: {}'.format(MIN_SCORE_ALIGN))

# Read index information:
index_list_fnam = 'index_list.xlsx'
index_df = pd.read_excel('{}/utils/{}'.format(homedir, index_list_fnam))

Notebook is in: /Users/krdav/Google Drive/MCB/Sullivan_lab/tRNA-charge-seq/projects/tRNAseq_third-gen
Repo is in: /Users/krdav/Google Drive/MCB/Sullivan_lab/tRNA-charge-seq
Using minimum read length: 49 (after merge)
Using minimum alignemnt score: 20


### Settings

In [4]:
sample_list_fnam = 'sample_list_P1-2-3.xlsx'
sample_df = pd.read_excel('{}/{}'.format(NBdir, sample_list_fnam))
# Add barcode sequences:
sample_df = index_to_sample_df(sample_df, index_df)
# Get filenames from the sample information:
inp_file_df = sample_df[['fastq_mate1_filename', 'fastq_mate2_filename', 'P5_index', 'P7_index', 'P5_index_seq', 'P7_index_seq']].copy().drop_duplicates().reset_index(drop=True)

# Downsample:
if True:
    sample_df, inp_file_df, seq_folder = downsample_raw_input(sample_df, inp_file_df, NBdir, data_folder, seq_folder_noDS, downsample_absolute=1e4)

In [5]:
# Run AdapterRemoval:
AR_obj = AR_merge(inp_file_df, NBdir, data_folder, seq_folder, AdapterRemoval_dir, MIN_READ_LEN)
AdapterRemoval_dir_abs = AR_obj.make_dir(overwrite=True)
# inp_file_df = AR_obj.run_serial()
inp_file_df = AR_obj.run_parallel()

In [6]:
# Split files based on barcodes:
BCsplit_obj = BC_split(sample_df, inp_file_df, NBdir, data_folder, AdapterRemoval_dir_abs, BC_dir)
BC_dir_abs = BCsplit_obj.make_dir(overwrite=True)
# sample_df, inp_file_df = BCsplit_obj.run_serial()
sample_df, inp_file_df = BCsplit_obj.run_parallel()

In [7]:
### Perform Kmer analysis on unmapped reads ###
if False: # barcode analysis is typically suficient 
    kmer_obj = Kmer_analysis(inp_file_df, index_df, BC_dir_abs)
    # Add a filter to avoid Kmers from the end of tRNA sequences:
    kmer_obj.filter_3p_fasta(tRNA_database['human'])
    # Add to this filter the constant region of the adapters:
    kmer_obj.filter_window_BC(filter_window=(0, 11))
    # Search for Kmers:
    all_kmer = kmer_obj.search_unmapped(search_size=13)

In [8]:
### Perform barcode analysis on unmapped reads ###
# Recall that adapters look like this:
# GGCTGCCATGC    GACTA
# GGCTGCCATGCA   AGTGC
# GGCTGCCATGCTG  TCACG
# GGCTGCCATGCAAC CTGAT
# With the barcode as the rightmost 5 nt.
# Search these 5 nt. barcodes by specifying BC_size_3p=5
bc_analysis_obj = BC_analysis(inp_file_df, index_df, BC_dir_abs, BC_size_3p=5)
# Search for barcodes in the unmapped reads.
# For the summary output, filter by a max distance
# to any barcode of 1 (group_dist=1), then group by barcode name:
bc_analysis_df = bc_analysis_obj.search_unmapped(group_dist=1)
bc_analysis_df

,Name,Count
0,l5Sp,163
1,l12Sp,108
2,l4Sp,65
3,l6Sp,49
4,l10Sp,48
5,l9Sp,48
6,l3Sp,42
7,l8Sp,39
8,l1Sp,37
9,l2Sp,31


In [9]:
### Generate UMI stats and write final trimmed tRNA sequences ###
# Note, the cDNA input amount is so large that it is very unlikely to sequence
# the same PCR amplified DNA twice. Therefore, this processing step does not
# attempt to merge possible UMI duplicates.
UMItrim_obj = UMI_trim(sample_df, NBdir, data_folder, BC_dir_abs, UMI_dir)
UMI_dir_abs = UMItrim_obj.make_dir(overwrite=True)
# sample_df = UMItrim_obj.run_serial()
sample_df = UMItrim_obj.run_parallel()
sample_df.head(3)

,sample_name_unique,sample_name,replicate,P5_index,P7_index,barcode,species,plot_group,hue_name,hue_value,hue_order,P5_index_seq,P7_index_seq,barcode_seq,fastq_mate1_filename,fastq_mate2_filename,N_total,N_CC,N_CCA,N_CCA+CC,CCA+CC_percent_total,percent_CCA,N_after_trim,N_UMI_observed,N_UMI_expected,percent_seqs_after_UMI_trim,percent_UMI_obs-vs-exp
0,100p1,100p,1,D501,D701,l1Sp,human,Charge-titration,Percent charge,100p,1,AGGCTATA,ATTACTCG,GGCTGCCATGCGACTA,P1_R1_DSA-10k.fastq.bz2,P1_R2_DSA-10k.fastq.bz2,942,38,901,939,99.681529,95.953142,929,865,928.178310,98.619958,93.193300
1,100p2,100p,2,D501,D702,l2Sp,human,Charge-titration,Percent charge,100p,1,AGGCTATA,TCCGGAGA,GGCTGCCATGCAAGTGC,P2_R1_DSA-10k.fastq.bz2,P2_R2_DSA-10k.fastq.bz2,673,14,648,662,98.365527,97.885196,663,621,662.581602,98.514116,93.724305
2,100p3,100p,3,D501,D703,l3Sp,human,Charge-titration,Percent charge,100p,1,AGGCTATA,CGCTCATT,GGCTGCCATGCTGTCACG,P3_R1_DSA-10k.fastq.bz2,P3_R2_DSA-10k.fastq.bz2,709,35,666,701,98.871650,95.007133,692,631,691.544180,97.602257,91.245074


In [14]:
### Align reads to database of reference tRNAs ###
align_obj = SWIPE_align(tRNA_database, sample_df, NBdir, data_folder, UMI_dir_abs, align_dir, SWIPE_score_mat, gap_penalty=6, extension_penalty=1, min_score_align=20)
align_dir_abs = align_obj.make_dir(overwrite=True)
# sample_df = align_obj.run_serial(dry_run=False, verbose=False, overwrite=False)
sample_df = align_obj.run_parallel()
sample_df.head(3)

Running Swipe on:  100p2  100p1  100p3  100p4  85p4  85p3  85p2  70p2  85p1  70p1  70p3  55p1  55p2  55p4  70p4  40p1  40p2  55p3  40p4  25p1  25p3  25p2  10p1  10p3  40p3  0p1  25p4  0p4  10p2  10p4  0p3  0p2  0p_90m3  0p_90m1  0p_90m2  0p_90m4

,sample_name_unique,sample_name,replicate,P5_index,P7_index,barcode,species,plot_group,hue_name,hue_value,hue_order,P5_index_seq,P7_index_seq,barcode_seq,fastq_mate1_filename,fastq_mate2_filename,N_total,N_CC,N_CCA,N_CCA+CC,CCA+CC_percent_total,percent_CCA,N_after_trim,N_UMI_observed,N_UMI_expected,percent_seqs_after_UMI_trim,percent_UMI_obs-vs-exp,N_mapped,percent_single_annotation,percent_multiple_annotation,Mapping_percent
0,100p1,100p,1,D501,D701,l1Sp,human,Charge-titration,Percent charge,100p,1,AGGCTATA,ATTACTCG,GGCTGCCATGCGACTA,P1_R1_DSA-10k.fastq.bz2,P1_R2_DSA-10k.fastq.bz2,942,38,901,939,99.681529,95.953142,929,865,928.178310,98.619958,93.193300,918,74.509804,25.490196,98.815931
1,100p2,100p,2,D501,D702,l2Sp,human,Charge-titration,Percent charge,100p,1,AGGCTATA,TCCGGAGA,GGCTGCCATGCAAGTGC,P2_R1_DSA-10k.fastq.bz2,P2_R2_DSA-10k.fastq.bz2,673,14,648,662,98.365527,97.885196,663,621,662.581602,98.514116,93.724305,652,74.079755,25.920245,98.340875
2,100p3,100p,3,D501,D703,l3Sp,human,Charge-titration,Percent charge,100p,1,AGGCTATA,CGCTCATT,GGCTGCCATGCTGTCACG,P3_R1_DSA-10k.fastq.bz2,P3_R2_DSA-10k.fastq.bz2,709,35,666,701,98.871650,95.007133,692,631,691.544180,97.602257,91.245074,673,72.956909,27.043091,97.254335
